# Distributed K-Means Testing

## Imports and basic functions

#### Imports libs needed and created function to get available GPUs and to make training data

In [1]:
import matplotlib.pyplot as plt

import threading

import os
import time

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.python.client import device_lib

from sklearn.datasets import make_classification

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

def make_data(filepath, n_obs, n_dim, seed, K):

    try:
        os.remove(filepath)
    except:
        print('file not found')
    finally:
        (X, Y) = make_classification(n_samples            = n_obs    , 
                                     n_features           = n_dim    ,
                                     n_informative        = n_dim    ,
                                     n_redundant          = 0        ,
                                     n_classes            = K        ,
                                     n_clusters_per_class = 1        ,
                                     shuffle              = True     ,
                                     class_sep            = 1.5      ,
                                     random_state         = seed      )
        
        np.savez(filepath, X=X, Y=Y)

    return True

#### Creates a Tensorflow InteractiveSession

An InteractiveSession is used to "debug" the code. It allows you to see what's happening. On production code, use Session instead.

In [2]:
# 

#### Defining variables used throughout the code

In [3]:
n_obs = 300000000
n_dim = 2
K     = 3
GPU_names = get_available_gpus()
n_max_iters = 20
seed = 800594

In [ ]:
make_data('test-data.npz', n_obs, n_dim, seed, K)

In [4]:
data = np.load('test-data.npz')

data_X = data['X']
data_Y = data['Y']
initial_centers = data_X[0:K, :]

#### Divide dataset into X equal parts, X being the number of available GPUs

In [5]:
# sizes = [len(arg) for arg in np.split( data_X[ (data_X.shape[0] % len(GPU_names)) :, :], len(GPU_names))]
sizes = [len(arg) for arg in np.split( data_X[ (data_X.shape[0] % 8) :, :], 8)]

partial_directions = []
partial_values = []
partial_results = []

#### Creates Tensorflow variable with the initial centroids

In [6]:
with tf.name_scope('global'):
    with tf.device('/cpu:0'):
        all_data = tf.placeholder(data_X.dtype, shape=(data_X.shape), name='all_data')
        print(all_data)

        dataX = tf.Variable(all_data)
        print(dataX)

        parts = tf.split(dataX, sizes, 0)
        print(tf.Variable(parts[0]))
        
        global_centroids = tf.Variable(initial_centers)

Tensor("global/all_data:0", shape=(300000000, 2), dtype=float64, device=/device:CPU:0)
<tf.Variable 'global/Variable:0' shape=(300000000, 2) dtype=float64_ref>
<tf.Variable 'global/Variable_1:0' shape=(37500000, 2) dtype=float64_ref>


#### For each GPU, calculate new local centroids

In [7]:
for GPU_num in range(len(GPU_names)):
    GPU_name = GPU_names[GPU_num]
        
    (X_mat) = parts[GPU_num]
    (N, M) = X_mat.get_shape().as_list()
    
    with tf.name_scope('scope_' + str(GPU_num)):
        with tf.device(GPU_name) :
            ####
            # In the coments we denote :
            # => N = Number of Observations
            # => M = Number of Dimensions
            # => K = Number of Centers
            ####
            # Data for GPU GPU_num to Clusterize
            X = tf.Variable(X_mat)

            # Reshapes rep_centroids and rep_points to format N x K x M so that 
            # the 2 matrixes have the same size
            rep_centroids = tf.reshape(tf.tile(global_centroids, [N, 1]), [N, K, M])
            rep_points = tf.reshape(tf.tile(X, [1, K]), [N, K, M])

            # Calculates sum_squares, a matrix of size N x K
            # This matrix is not sqrt((X-Y)^2), it is just(X-Y)^2
            # Since we need just the argmin(sqrt((X-Y)^2)) wich is equal to 
            # argmin((X-Y)^2), it would be a waste of computation
            sum_squares = tf.reduce_sum(tf.square(tf.subtract( rep_points, rep_centroids) ), axis = 2)

            # Use argmin to select the lowest-distance point
            # This gets a matrix of size N x 1
            best_centroids = tf.argmin(sum_squares, axis = 1)
            
            means = []
            for c in range(K):
                means.append(
                    tf.reduce_mean(
                        tf.gather(X, tf.reshape(tf.where(tf.equal(best_centroids, c)), [1,-1])), axis=[1]))

            new_centroids = tf.concat(means, 0)
#             print('GPU: ', GPU_name)
#             print('Initial centers ', initial_centers)
#             print('New centroids ', new_centroids.eval())
                
        with tf.device('/cpu:0'):
            y_count = tf.cast(
                tf.bincount(tf.to_int32(best_centroids), maxlength = K, minlength = K), dtype = tf.float64)
            
            partial_mu =  tf.multiply( tf.transpose(new_centroids), y_count )

            partial_directions.append( y_count )
            partial_values.append( partial_mu )

#### After local GPU centroids have been calculated, create new global centroids

In [8]:
with tf.name_scope('global') :
    with tf.device('/cpu:0') :
        sum_direction = tf.add_n( partial_directions )
        sum_mu = tf.add_n( partial_values )

        rep_sum_direction = tf.reshape(tf.tile(sum_direction, [M]), [M, K])
        new_centers = tf.transpose( tf.div(sum_mu, rep_sum_direction) )

        update_centroid = tf.group( global_centroids.assign(new_centers) )

In [9]:
config = tf.ConfigProto( allow_soft_placement = True )
config.gpu_options.allow_growth = True
config.gpu_options.allocator_type = 'BFC'

sess = tf.Session( config = config )

init = tf.global_variables_initializer()

print(data_X.dtype, data_X.shape)

float64 (300000000, 2)


In [10]:
sess.run(init, feed_dict={all_data: data_X})

ResourceExhaustedError: OOM when allocating tensor with shape[37500000,2]
	 [[Node: scope_0/Variable/Assign = Assign[T=DT_DOUBLE, _class=["loc:@scope_0/Variable"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/gpu:0"](scope_0/Variable, scope_0/Variable/global/split_scope_0/Variable_0)]]

Caused by op 'scope_0/Variable/Assign', defined at:
  File "/home/enacom/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/enacom/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/enacom/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/enacom/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/enacom/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().s

In [ ]:
for i in range(n_max_iters):
    [result, _] = sess.run([global_centroids, update_centroid])

### Plotting graphs to evaluate results

#### Initial centroids

In [ ]:
plt.scatter(data_X[0:np.min([data_X.shape[0], 10000]), 0], 
            data_X[0:np.min([data_X.shape[0], 10000]), 1], 
            c = data_Y[0:np.min([data_X.shape[0], 10000])],
            alpha = 0.8, marker = (5, 2))
plt.scatter(initial_centers[:, 0], initial_centers[:, 1], alpha = 1, c = 'red', marker = (5, 3))
plt.show()

#### After computation

In [ ]:
plt.scatter(data_X[0:np.min([data_X.shape[0], 10000]), 0], 
            data_X[0:np.min([data_X.shape[0], 10000]), 1], 
            c = data_Y[0:np.min([data_X.shape[0], 10000])],
            alpha = 0.8, marker = (3, 1))
plt.scatter(result[:, 0], result[:, 1], alpha = 1, c = 'blue', marker = (5, 3))
plt.show()